# **Lab04: Stack/Queue; Heap**

- ### Your Name:
Jiarui HE

- ### Your Student ID:
50013538

## Task 1. Stacks/Queues

Show how to implement a queue using two stacks in Python.

**TODO: Please finish the given tasks in the code block below:** (20% Marks)

Hits: You can consider to use some of the operations in Python for stacks like append(), pop() when editing the function enqueue() and dequeue(), and the function print_list() is for debugging, you can use some Array operation like reversed() to edit your idea.

## Introduction

In the upcoming lab, you will be tasked with the implementation of the queue data structure using two stacks. Additionally, you will have the opportunity to gain an understanding of the heap (priority queue) and its functionality in a particular scenario involving a single-threaded CPU.

Please **note** that NO LATE SUBMISSION will be accepted. If you truly has extraordinary conditions that prevent you from submitting on time, please send email with evidence to  *Professor WANG* and the TA of your experimental class. 

### Goal:
1. To acquire a comprehensive understanding of Stack and Queue data structures.
2. To acquire practical problem-solving skills utilizing 'heap' data structure in a realistic scenario.

In [22]:
class QueueUsingStacks:
    def __init__(self):
        self.dtStk, self.swpStk = [], []

    def enqueue(self, element):
        self.dtStk.append(element)

    def dequeue(self):
        if len(self.dtStk) == 0: return
        while len(self.dtStk) > 0:
            self.swpStk.append(self.dtStk.pop())
        self.swpStk.pop()
        while len(self.swpStk) > 0:
            self.dtStk.append(self.swpStk.pop())
    
    def print_list(self):
        while len(self.dtStk) > 0:
            self.swpStk.append(self.dtStk.pop())
        while len(self.swpStk) > 0:
            print(self.swpStk[-1], end=' ')
            self.dtStk.append(self.swpStk.pop())
        print()  


# Testing Usages
queue = QueueUsingStacks()

queue.enqueue(1)
queue.enqueue(2)
queue.enqueue(3)
queue.print_list()  # Output: 1 2 3

queue.dequeue()
queue.print_list()  # Output: 2 3

queue.enqueue(4)
queue.print_list()  # Output: 2 3 4

1 2 3 
2 3 
2 3 4 


## Task 2. Sequence N -> M
We are given numbers n and m, and the following operations:

(a)	n -> n + 1

(b)	n -> n + 2

(c)	n -> n * 2

Write a program that finds all the distinct sequence of operations within k steps from the list above that starts from n and finishes in m.

Examples:

|  Input(n, m, k)   | Output  |
|  ----  | ----  |
| 3 10 3 | 3->5->10; 3->4->5->10; 3->4->8->10; 3->6->8->10 |
| -5 10 10  | No sequences for 5 to -5 within 10 steps |
| 10 30 3 | No sequences for 10 to 30 within 3 steps | 


Hint: use a queue to finish this task. And you are allowed to use deque.append() and deque.popleft() (https://docs.python.org/3/library/collections.html#collections.deque) to simulate the data structure queue.


You may find the algorithm either will find a solution, or will find that it does not exist. But it cannot print the numbers comprising the sequence n -> m.


To print the sequence of steps to reach m, starting from n, you will need to keep the previous item as well. Instead using a queue of numbers, use a queue of items. Each item will keep a number and a pointer to the previous item.


**TODO: Please follow the problem to fill in your code answer below. (30% Marks)**

In [23]:
# All the codes below and your correct and reasonable outputs will be evaluated. (30 Marks)

from collections import deque

class Item:
    def __init__(self, data : int, prev = None) :
        self.data = data
        self.prev = prev

def print_solution(item : Item):
    stk = []
    while item != None:
        stk.append(item.data)
        item = item.prev
    while len(stk) :
        print(stk.pop(), end='')
        if len(stk) > 0:
            print("->",end='')
    print()
solQue = deque()
curItem : Item = None
def find_all_sequences(n, m, k):
    global curItem
    def recur(n, m, k) : 
        global curItem
        if n == m: solQue.append(curItem)
        if k == 0: return
        lstItem = curItem
        curItem = Item(lstItem.data + 1, lstItem)
        recur(n + 1, m, k - 1)
        curItem = Item(lstItem.data + 2, lstItem)
        recur(n + 2, m, k - 1)
        if n + 2 != n * 2 and n + 1 != n * 2 :
            curItem = Item(lstItem.data * 2, lstItem)
            recur(n * 2, m, k - 1)

    while len(solQue) > 0: solQue.popleft()
    curItem = Item(n)
    recur(n, m, k)
    if len(solQue) == 0:
        print(f"No sequences for {n} to {m} within {k} steps")
    while len(solQue) > 0: 
        print_solution(solQue.popleft())

# Testing the code
test_cases = [
    (3, 10, 3),   # n, m, k
    (5, -5, 10),
    (10, 30, 3),
    (0, 16, 4),
    (7, 28, 5)
]

# TODO: Your codes for your test_cases to get the correct output
for case in test_cases:
    find_all_sequences(case[0], case[1], case[2])

3->4->5->10
3->4->8->10
3->5->10
3->6->8->10
No sequences for 5 to -5 within 10 steps
No sequences for 10 to 30 within 3 steps
0->2->4->8->16
7->8->10->12->14->28
7->9->10->12->14->28
7->9->11->12->14->28
7->9->11->13->14->28
7->9->11->13->26->28
7->14->28


### Bonus Part of Test 2: (20% Marks)

**The previous solution can be inefficient for large k values because it explores all possible paths, can you design a new and more effcient one for this problem? (You can use all the method you have learned) (15% Marks) And try to use simple words to explain why it works. (Otherwise all your answers may not be considered valid.) (5% Marks)** 

In [24]:
import copy
import math
from queue import PriorityQueue

class SolLst :
    def __init__(self) :
        self.lst : list[(int, str)] = []
    def merge(self, other, lim = -1) :
        if other == None: return
        if lim == -1: self.lst += other.lst; return
        l, r = 0, len(other.lst) - 1
        while l <= r:
            mid = (l + r) >> 1
            if other.lst[mid][0] <= lim: l = mid + 1
            else : r = mid - 1
        self.lst += other.lst[0 : l]
    def append(self, data : int) :
        for i in range(0, len(self.lst)) :
            self.lst[i] = (self.lst[i][0] + 1, self.lst[i][1] + f"->{data}")
    def add(self, data : int):
        self.lst.append((0, str(data)))
    def __str__(self) : 
        res : str = ''
        for (_, seq) in self.lst: res += seq + '\n'
        return res
    def rearrange(self):
        self.lst.sort()

def find(n : int, m : int, limK : int) -> SolLst :
    if n > m: return None
    fMap : dict[int, int] = {}
    visMap = set()
    def updf(pos : int, vl : int) -> bool:
        if pos not in fMap : fMap[pos] = vl; return True
        else :
            if fMap[pos] > vl: fMap[pos] = vl; return True
            else : return False
    que = PriorityQueue()
    que.put((0, m))
    fMap[m] = 0
    while not que.empty():
        (k, u) = que.get()
        if u in visMap or k == limK or u < 0: continue
        visMap.add(u)
        vl = fMap[u]
        if updf(u - 1, vl + 1) and u - 1 not in visMap: que.put((vl + 1, u - 1))
        if updf(u - 2, vl + 1) and u - 2 not in visMap : que.put((vl + 1, u - 2))
        if u % 2 == 0 and updf(u // 2, vl + 1) and u // 2 not in visMap: que.put((vl + 1, u // 2))
        
    if n not in fMap: return None
    solMap : dict[int, SolLst] = {}
    lst = SolLst()
    if n == 0:
        s = "0"
        dis = fMap[0]
        for i in range(0, limK - dis + 1):
            lst.lst.append((i, s))
            s += "->0"
    else: lst.add(n)
    solMap[n] = lst

    def upd(m0, insM) :
        if not (m0 in fMap) : return
        if not (m0 in solMap): solMap[m0] = SolLst()
        solMap[m0].merge(insM, limK - fMap[m0] - 1)
    for i in range(n, m):
        if not (i in solMap): continue
        if i != n: solMap[i].append(i)
        solMap[i].rearrange()
        upd(i + 1, solMap[i]),
        upd(i + 2, solMap[i])
        if i + 1 != i * 2 and i + 2 != i * 2 : upd(i * 2, solMap[i])
    if not (m in solMap): return None
    else:
        solMap[m].append(m) 
        return solMap[m]
        
# Testing the code
test_cases = [
    (3, 10, 3),   # n, m, k
    (5, -5, 10),
    (10, 30, 3),
    (0, 16, 4),
    (7, 28, 5)
]

# TODO: Your codes for your test_cases to get the correct output
for (n, m, k) in test_cases:
    if n >= 0 : 
        lst = find(n, m, k)
        if lst == None or len(lst.lst) == 0:
            print(f"No sequences for {n} to {m} within {k} steps")
        else : 
            for seq in lst.lst: print(seq[1])
    else :
        find_all_sequences(n, m, k)

3->5->10
3->4->5->10
3->4->8->10
3->6->8->10
No sequences for 5 to -5 within 10 steps
No sequences for 10 to 30 within 3 steps
0->2->4->8->16
7->14->28
7->8->10->12->14->28
7->9->10->12->14->28
7->9->11->12->14->28
7->9->11->13->14->28
7->9->11->13->26->28


### Your Explaination of your Codes: (Please use more keywords as briefly as possible to describe your thinking.)
Actually I can only write a program that handle with the cases that $n \ge 0$ , since that the conditions and process of $n<0$ is much more complex, and I call the previous program to handle with $n < 0$ .

I use the reverse process for constructing the sequence : $ + 1\Rightarrow - 1$, $+2 \Rightarrow - 2$, $\times 2\Rightarrow\times \frac{1}{2}$

I use a $\texttt{dict}$ called $\texttt{solMap}$ to store the the pairs of the length of the prefix and the prefix itself for the every state $m$ and sort these pairs by length.

Considering the reverse process, when we reach to number $v$, we get a "current path" which looks like $v\rightarrow...\rightarrow m$ , and the length of one path is the number of $\rightarrow$. Since that the set of valid prefixes for the longer current path is the subset of the valid prefixes for a shorter current path, I use $\texttt{dijsktra}$ to calculate the shortest path for each number $u$, called $f(u)$. Then merge the paths from $u$ to $u+1, u+2, u\times 2$ while $u$ is considered from $n$ to $m-1$ , when we merge the paths in $u$ into $v$, we only need the paths whose lengths are less or equal to $k - f(v) - 1$ , and add $v$ to the end of the new paths.

I am not able to calculate the exact time efficience of this program. But I am sure that this program is faster than the previous.


## Grading Policy

The marks of this is lab is composed of:

* Submission: 50%

* Task1: 20%

* Task2: 30%

* Bonus: 20%

    * Codes: 15%
    
    * Explaination: 5%